In [1]:
!pip install ffprobe-python

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


### FFProbeWrapper

The rapper looks for the ffprobe on local machine and passes the arguments of "ffprobe -show_stream". The wrapper will check to see if the process exists on local machine through use of the command "ffprobe -h".

### Reference
[1] https://github.com/gbstack/ffprobe-python

In [77]:
import functools
import operator
import os
import pipes
import platform
import re
import subprocess


class FFProbeWrapper:
    """
    FFProbeWrapper wraps the ffprobe command and pulls the data into an object
    """

    def __init__(self, path):
        
        #path to file
        self.file_path = path

        try:
            with open(os.devnull, 'w') as file:
                subprocess.check_call(["ffprobe", "-h"], stdout=file, stderr=file)
        except FileNotFoundError:
            raise IOError('ffprobe was not found.')

        if os.path.isfile(self.file_path) or self.file_path.startswith('http'):
            if platform.system() == 'Windows':
                command = ["ffprobe", "-show_streams", self.file_path]
            else:
                command = ["ffprobe -show_streams " + pipes.quote(self.file_path)]

            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        

            stream = False
            ignore_line = False
            
            #setup variables 
            self.streams = []
            self.video = []
            self.audio = []

            for line in iter(process.stdout.readline, b''):
                line = line.decode('UTF-8', 'ignore')
                

                if '[STREAM]' in line:
                    stream = True
                    ignore_line = False
                    data = []
                elif '[/STREAM]' in line and stream:
                    stream = False
                    ignore_line = False
                    self.streams.append(StreamFF(data))
                elif stream:
                    if '[SIDE_DATA]' in line:
                        ignore_line = True
                    elif '[/SIDE_DATA]' in line:
                        ignore_line = False
                    elif ignore_line == False:
                        data.append(line)

            self.metadata = {}
            is_metadata = False
            stream_metadata = False

            for line in iter(process.stderr.readline, b''):
                line = line.decode('UTF-8', 'ignore')

                if 'Metadata:' in line and not stream_metadata:
                    is_metadata = True
                elif 'Stream #' in line:
                    is_metadata = False
                    stream_metadata = True
                elif is_metadata:
                    splits = line.split(',')
                    for split in splits:
                        labels = re.search(r'(\w+)\s*:\s*(.*)$', split)
                        if labels is not None:
                 
                            self.metadata[labels.groups()[0]] = labels.groups()[1].strip()

                if '[STREAM]' in line:
                    stream = True
                    data = []
                elif '[/STREAM]' in line and stream:
                    stream = False
                    self.streams.append(StreamFF(data))
                elif stream:
                    data.append(line)

            process.stdout.close()
            process.stderr.close()
            
            
            #Get audio and video streams
            for stream in self.streams:
                if stream.audio_type():
                    self.audio.append(stream)
                elif stream.video_type():
                    self.video.append(stream)

        else:
            raise IOError('Input Error for file in: ' + self.file_path)

    

class StreamFF:
    """
    An object representation of a single stream.
    """

    def __init__(self, data):
        for line in data:
            self.__dict__.update({key: value for key, value, *_ in [line.strip().split('=')]})

            try:
                self.__dict__['framerate'] = round(
                    functools.reduce(
                        operator.truediv, map(int, self.__dict__.get('avg_frame_rate', '').split('/'))
                    )
                )

            except ValueError:
                self.__dict__['framerate'] = None
            except ZeroDivisionError:
                self.__dict__['framerate'] = 0
                
    
    def audio_type(self):
        """
        Determines if Stream is an audio stream
        """
        return self.__dict__.get('codec_type', None) == 'audio'

    def video_type(self):
        """
        Determines if Stream is a video stream.
        """
        return self.__dict__.get('codec_type', None) == 'video'

In [130]:
# Local file
metadata=FFProbeWrapper('films/The_Hill_Gang_Rides_Again.mp4')

video = metadata.video[0]
audio = metadata.audio[0]
print('Video format (container): {}.'.format(video.__dict__.get('codec_long_name', '')))
print('Video codec: {}'.format(video.__dict__.get('codec_long_name')))
print('Audio codec: {}'.format(audio.__dict__.get('codec_name', '')))
print('Frame rate: {} fps'.format(video.__dict__.get('framerate', '')))
print('Aspect Ratio: {}'.format(video.__dict__.get('display_aspect_ratio', '')))

width = video.__dict__.get('coded_width', '')
height = video.__dict__.get('coded_height', '')

print('Resolution: {} x {}'.format(width, height))
print('Video bit rate: {} kb/s'.format(int(video.__dict__.get('bit_rate', ''))/1000))
print('Audio bit rate: {} kb/s'.format(int(audio.__dict__.get('bit_rate', ''))/1000))
print('Audio channels: {}'.format(audio.__dict__.get('channels', '')))

Video format (container): H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10.
Video codec: H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
Audio codec: aac
Frame rate: 25 fps
Aspect Ratio: 16:9
Resolution: 640 x 360
Video bit rate: 7537.73 kb/s
Audio bit rate: 253.272 kb/s
Audio channels: 2


In [120]:
import os.path
import os
import re
from subprocess import Popen, PIPE
import logging
import locale


#logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


class ErrorFFMpeg(Exception):
    pass


class ConversionErrorFFMpeg(Exception):
    def __init__(self, message, command, output, details=None, pid=0):
        """
        Returns Conversion Error Details and Conversion Message
        """
        
        super(ConversionErrorFFMpeg, self).__init__(message)

        self.command = command
        self.output = output
        self.details = details
        self.pid = pid

    def __repr__(self):
        return ('<ConversionErrorFFMpeg details="{}", pid={}, command="{}", message="{}">'
                .format(self.details, self.pid, self.command, self.message))

    def __str__(self):
        return self.__repr__()



class FFMpegWrapper(object):
    """
    FFMPeg wrapper object
    """

    def __init__(self, binary_path=None):
        """
        Initialize a new FFMpeg wrapper object.
        Class attempts to find local path to ffmpeg
        """

        def FindLocalPath(name):
            path = os.environ.get('PATH', os.defpath)
            for dd in path.split(':'):
                file_path = os.path.join(dd, name)
                if os.path.exists(file_path) and os.access(file_path, os.X_OK):
                    return file_path
            return None

        if binary_path is None:
            binary_path = 'ffmpeg'

        if '/' not in binary_path:
            binary_path = FindLocalPath(binary_path) or binary_path


        self.binary_path = binary_path

        if not os.path.exists(self.binary_path):
            raise ErrorFFMpeg("ffmpeg binary not found: " + self.binary_path)


    @staticmethod
    def execute(commands):
        logger.debug('Executing ffmpeg with command: ' + ' '.join(commands))
        return Popen(commands, shell=False, stdin=PIPE, stdout=PIPE, stderr=PIPE,
                     close_fds=True)


    def convert(self, in_file, out_file, options):
        """
        Convert the source media (in_file)
        """

        if not os.path.exists(in_file):
            raise ErrorFFMpeg("Input file does not exist: " + in_file)

        commands = [self.binary_path, '-i', in_file]
        commands.extend(options)
        commands.extend(['-y', out_file])
        


        try:
            process = self.execute(commands)
        except OSError:
            raise ErrorFFMpeg('Error while calling ffmpeg binary')

        succeeded = False
        buffer = ''
        output_result = ''
        pattern = re.compile(r'time=([0-9.:]+) ')
        while True:
            
            returned = process.stderr.read(10)

            if not returned:
                break
            
            returned = returned.decode(locale.getdefaultlocale()[1] or 'UTF-8')
            output_result += returned
            buffer += returned
            if '\r' in buffer:
                line, buffer = buffer.split('\r', 1)

                time = pattern.findall(line)
                print(time)
                
                if len(time) == 1:
                    timespec = time[0]
                    if ':' in timespec:
                        timecode = 0
                        for part in timespec.split(':'):
                            timecode = 60 * timecode + float(part)
                    else:
                        timecode = float(time[0])
                        
                    succeeded = True
                    yield timecode


        process.communicate()  # wait for process to exit

        if output_result == '':
            raise ErrorFFMpeg('An Error occured while calling ffmpeg')
            

        command = ' '.join(commands)
        
        if '\n' in output_result:
            line = output_result.split('\n')[-2]

            if line.startswith('Received signal'):
                raise ConversionErrorFFMpeg(line.split(':')[0], command, output_result, pid=process.pid)
            if line.startswith(in_file + ': '):
                err = line[len(in_file) + 2:]
                raise ConversionErrorFFMpeg('Encoding error', command, output_result, err, pid=process.pid)
            if line.startswith('Error while '):
                raise ConversionErrorFFMpeg('Encoding error', command, output_result, line, pid=process.pid)
            if not succeeded:
                raise ConversionErrorFFMpeg('Unknown ffmpeg error', command,output_result, line, pid=process.pid)
        
        #check process return code        
        if process.returncode != 0:
            raise ConversionErrorFFMpeg('Exited with code {}'.format(process.returncode), command, output_result, pid=process.pid)

            
            

## Codec Classes:

Creating Codec Classes for all supported video/audio codecs and all supported formats

In [121]:
class BaseCodec(object):
    """
    Base audio/video codec class.
    """

    encoder_options = {}
    name = None
    ffmpeg_name = None

    def ConsumeOptions(self, option):
        if 'codec' not in option or option['codec'] != self.name:
            raise ValueError('wrong codec name')
        return None

    def ConsumeFFmpegOptions(self, sf):
        return sf

    def GenerateCodecList(self, sf):
        return []

    def sf_options(self, options):
        sf = {}

        # copy options and do typecasting 
        for key, value in options.items():
            if key in self.encoder_options:
                typ = self.encoder_options[key]
                try:
                    sf[key] = typ(value)
                except:
                    pass

        return sf


# Audio Codecs
class AudioCodec(BaseCodec):
    """
    Base AudioCodec
    """

    encoder_options = {
        'codec': str,
        'channels': int,
        'bitrate': int,
        'samplerate': int
    }

    def ConsumeOptions(self, options):
        super(AudioCodec, self).ConsumeOptions(options)

        sf = self.sf_options(options)

        if 'channels' in sf:
            channels = sf['channels']
            if channels < 1 or channels > 12:
                del sf['channels']

        if 'bitrate' in sf:
            bitrate = sf['bitrate']
            if bitrate < 8 or bitrate > 512:
                del sf['bitrate']

        if 'samplerate' in sf:
            samplerate = sf['samplerate']
            if samplerate < 1000 or samplerate > 50000:
                del sf['samplerate']

        sf = self.ConsumeFFmpegOptions(sf)

        optionsList = ['-acodec', self.ffmpeg_name]
        if 'channels' in sf:
            optionsList.extend(['-ac', str(sf['channels'])])
        if 'bitrate' in sf:
            optionsList.extend(['-ab', str(sf['bitrate']) + 'k'])
        if 'samplerate' in sf:
            optionsList.extend(['-ar', str(sf['samplerate'])])

        optionsList.extend(self.GenerateCodecList(sf))
        return optionsList

class AudioCopyCodec(BaseCodec):
    """
    Copy audio stream directly from the source.
    """
    name = 'copy'

    def ConsumeOptions(self, option):
        return ['-acodec', 'copy']
    
    
class AudioNullCodec(BaseCodec):
    """
    Handles no audio.
    """
    name = None

    def ConsumeOptions(self, option):
        return ['-an']
    


class AacCodec(AudioCodec):
    """
    AAC audio codec.
    """
    name = 'aac'
    ffmpeg_name = 'aac'
    experimental_aac = ['-strict', 'experimental']

    def GenerateCodecList(self, sf):
        return self.experimental_aac



class Mp3Codec(AudioCodec):
    """
    MP3 (MPEG layer 3) audio codec.
    """
    name = 'mp3'
    ffmpeg_name = 'libmp3lame'

    

#Put codecs into list
supported_audio_codec = [
    AudioNullCodec, AudioCopyCodec, AacCodec, Mp3Codec
]


# Video Codecs
class VideoCodec(BaseCodec):
    """
    Base video codec class
    """

    encoder_options = {
        'codec': str,
        'bitrate': int,
        'fps': int,
        'width': int,
        'height': int,
        'mode': str,
        'src_width': int,
        'src_height': int,
    }

    def CorrectAspects(self, src_width, src_height, width, height, mode):
        
        # check if src width and height exist,
        if not src_width or not src_height:
            return width, height, None

        # Original aspect ratio
        aspect = (1.0 * src_width) / (1.0 * src_height)

        # If we have only one dimension
        if not width and not height:
            return width, height, None
        elif width and not height:
            height = int((1.0 * width) / aspect)
            return w, h, None
        elif height and not width:
            width = int(aspect * height)
            return width, height, None

        # If source == target dimensions 
        if int(aspect * height) == width:
            return width, height, None
        
        #stretch mode
        if mode == 'stretch':
            return width, height, None

        target_aspect = (1.0 * width) / (1.0 * height)
        
        #crop mode
        if mode == 'crop':
            # if source is taller
            if target_aspect > aspect:  # target is taller
                height_crop = int(width / aspect)
                assert height_crop > height, (src_width, src_height, width, height)
                cropped_height = (height_crop - height) / 2
                return width, height_crop, 'crop={}:{}:0:{}'.format(width, height, cropped_height)
            else:  #if source is wider
                width_crop = int(height * aspect)
                assert width_crop > width, (src_width, src_height, width, height)
                cropped_width = (width_crop - width) / 2
                return width_crop, height, 'crop={}:{}:{}:0'.format(width, height, cropped_width)
        
        #pad mode
        if mode == 'pad':
            # target is taller
            if target_aspect < aspect:
                height_pad = int(width / aspect)
                assert height_pad < height, (src_width, src_height, width, height)
                padded_height = (height - height_pad) / 2
                return width, height_pad, 'pad={}:{}:0:{}'.format(width, height, padded_height)  
            else:  # target is wider
                width_pad = int(height * aspect)
                assert width_pad < width, (src_width, src_height, width, height)
                padded_width = (width - width_pad) / 2
                return width_pad, height, 'pad={}:{}:{}:0'.format(width, height, padded_width)  

        assert False, mode

    def ConsumeOptions(self, options):
        super(VideoCodec, self).ConsumeOptions(options)

        sf = self.sf_options(options)

        if 'fps' in sf:
            fps = sf['fps']
            if fps < 1 or fps > 120:
                del sf['fps']

        if 'bitrate' in sf:
            bitrate = sf['bitrate']
            if bitrate < 16 or bitrate > 15000:
                del sf['bitrate']

        width = None
        height = None

        if 'width' in sf:
            width = sf['width']
            if width < 16 or width > 4000:
                width = None

        if 'height' in sf:
            height = sf['height']
            if height < 16 or height > 3000:
                height = None
        
        ## Source width and height
        src_width = None
        src_height = None

        if 'src_width' in sf and 'src_height' in sf:
            src_width = sf['src_width']
            src_height = sf['src_height']
            if not src_width or not src_height:
                src_width = None
                src_height = None

        mode = 'stretch'
        if 'mode' in sf:
            if sf['mode'] in ['stretch', 'crop', 'pad']:
                mode = sf['mode']

        original_width, original_height = width, height  # FIXED
        width, height, filters = self.CorrectAspects(src_width, src_height, width, height, mode)

        sf['width'] = width
        sf['height'] = height
        sf['aspect_filters'] = filters

        if width and height:
            sf['aspect'] = '{}:{}'.format(width, height)

        sf = self.ConsumeFFmpegOptions(sf)

        width = sf['width']
        height = sf['height']
        filters = sf['aspect_filters']

        optionsList = ['-vcodec', self.ffmpeg_name]
        
        if 'fps' in sf:
            optionsList.extend(['-r', str(sf['fps'])])
        if 'bitrate' in sf:
            optionsList.extend(['-vb', str(sf['bitrate']) + 'k']) 
        if width and height:
            optionsList.extend(['-s', '{}x{}'.format(width, height)])

            if original_width and original_height:
                optionsList.extend(['-aspect', '{}:{}'.format(original_width, original_height)])

        if filters:
            optionsList.extend(['-vf', filters])

        optionsList.extend(self.GenerateCodecList(sf))
        return optionsList


class VideoCopyCodec(BaseCodec):
    """
    Copy video stream from source.
    """
    name = 'copy'
    
    def ConsumeOptions(self, option):
        return ['-vcodec', 'copy']
    
class VideoNullCodec(BaseCodec):
    """
    Null video codec, i.e. no video.
    """

    name = None

    def ConsumeOptions(self, option):
        return ['-vn']


class H264Codec(VideoCodec):
    """
    H.264 video codec.
    """
    name = 'h264'
    ffmpeg_name = 'libx264'
    
    encoder_options = VideoCodec.encoder_options.copy()
    encoder_options.update({
        'preset': str, 
        'quality': int,  
        'profile': str, 
        'tune': str,  
    })

    def GenerateCodecList(self, sf):
        optionsList = []
        if 'preset' in sf:
            optionsList.extend(['-preset', sf['preset']])
        if 'quality' in sf:
            optionsList.extend(['-crf', sf['quality']])
        if 'profile' in sf:
            optionsList.extend(['-profile', sf['profile']])
        if 'tune' in sf:
            optionsList.extend(['-tune', sf['tune']])
        return optionsList


class H263Codec(VideoCodec):
    """
    H.263 video codec.
    """
    name = 'h263'
    ffmpeg_name = 'h263'   

#supported video codec
supported_video_codec = [VideoNullCodec, VideoCopyCodec, H264Codec, H263Codec]



#BaseFormat for Supported formats
class BaseFormat(object):
    """
    Base format class.
    Supported formats: mov, mp4, mp3, avi
    """

    name = None
    ffmpeg_name = None

    def ConsumeOptions(self, option):
        if 'format' not in option or option.get('format') != self.name:
            raise ValueError('invalid Format')
            
        return ['-f', self.ffmpeg_name]



class AviFormat(BaseFormat):
    """
    Avi container format, used vith DivX video.
    """
    name = 'avi'
    ffmpeg_name = 'avi'


class MovFormat(BaseFormat):
    """
    Mov container format, used with H.264 video
    """
    name = 'mov'
    ffmpeg_name = 'mov'


class Mp4Format(BaseFormat):
    """
    Mp4 container format. The default Format for H.264
    """
    name = 'mp4'
    ffmpeg_name = 'mp4'


class Mp3Format(BaseFormat):
    """
    Mp3 container
    """
    name = 'mp3'
    ffmpeg_name = 'mp3'


## Supported formats list
supported_formats = [AviFormat, MovFormat, Mp4Format, Mp3Format]


In [122]:
import ffmpeg
import numpy as np
import ffmpeg

class ConversionError(Exception):
    '''Conversion error'''
    pass


class Converter(object):
    """
    Converter class use as, b = Converter()
    """

    def __init__(self, in_file, ffmpeg_path=None):
        """
        Initialize a new Converter object to convert files.
        """

        self.ffmpeg = FFMpegWrapper(binary_path=ffmpeg_path)
        self.ffprobe = FFProbeWrapper(in_file)
        self.video_codecs = {}
        self.audio_codecs = {}
        self.formats = {}
        
        #audio codec
        for audio_codec in supported_audio_codec:
            name = audio_codec.name
            self.audio_codecs[name] = audio_codec
        
        #video codec
        for video_codec in supported_video_codec:
            name = video_codec.name
            self.video_codecs[name] = video_codec
        
        #formats
        for file_format in supported_formats:
            name = file_format.name
            self.formats[name] = file_format

    def ConsumeOptions(self, options):
        """
        Parse format/codec options and prepare option list.
        Returns: optionsList
        """
        
        if not isinstance(options, dict):
            raise ConversionError('Invalid output file specification')
        
        # Format options
        if 'format' not in options:
            raise ConversionError('Format has not been specified')

        file_format = options['format']
        if file_format not in self.formats:
            raise ConversionError('Requested to convert to an unknown format: ' + str(file_format))

        format_options = self.formats[file_format]().ConsumeOptions(options)
        
        if format_options is None:
            raise ConversionError('Unknown container format error occured')
        
        #Check if audio or video streams are requested
        if 'audio' not in options and 'video' not in options:
            raise ConversionError('Neither audio nor video streams requested')

        # audio options
        if 'audio' not in options:
            option_audio = {'codec': None}
        else:
            option_audio = options['audio']
            if not isinstance(option_audio, dict) or 'codec' not in option_audio:
                raise ConversionError('Invalid audio codec specification requested')

        codec = option_audio['codec']
        if codec not in self.audio_codecs:
            raise ConversionError('Requested an unknown audio codec ' + str(c))
            
        audio_options = self.audio_codecs[codec]().ConsumeOptions(option_audio)
        
        if audio_options is None:
            raise ConversionErrorError('Unknown audio codec error occured')

        # video options
        if 'video' not in options:
            option_video = {'codec': None}
        else:
            option_video = options['video']
            if not isinstance(option_video, dict) or 'codec' not in option_video:
                raise ConversionError('Invalid video codec specification')

        codec = option_video['codec']
        if codec not in self.video_codecs:
            raise ConversionError('Requested unknown video codec ' + str(codec))

        video_options = self.video_codecs[codec]().ConsumeOptions(option_video)
        
        if video_options is None:
            raise ConversionError('Unknown video codec error occured')


        # Combine all options
        optionsList = audio_options + video_options + format_options

        
        #return options list
        return optionsList

    def convert(self, in_file, out_file, options):
        '''Converts files to mp4'''
        

        if not isinstance(options, dict):
            raise ConversionError('Invalid options')

        if not os.path.exists(in_file):
            raise ConversionError("Source file doesn't exist: " + in_file)


        optionsList = self.ConsumeOptions(options)
        for timecode in self.ffmpeg.convert(in_file, out_file, optionsList):
            yield int((50.0 * timecode) / float(self.ffprobe.video[0].__dict__.get('duration', '')))
            

## Convert_To_MP4:

This function converts any video file into the specifications of an audio format of mp4, audio codec of 'aac' with a 256 kb/s bitrate and 2 audio channels. Moreover, the video codec is converted to h265 with a resolution of 640 x 360, 25 frames per second and a 2.56 Mb/s video bit rate.

In [138]:
path1= 'films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4'
path2= 'films/Cosmos_War_of_the_Planets.mp4'
path3= 'films/Last_man_on_earth_1964.mov'
path4= 'films/The_Gun_and_the_Pulpit.avi'
path5= 'films/The_Hill_Gang_Rides_Again.mp4'

files = [path1, path2,path3,path4,path5]

def ConvertToMP4(path, file):
    
    ffmpeg = Converter(path)
    
    converted_name = path.split('.')[0] + '_formatOK' + '.mp4'
    original_name = path.split('/')[-1]
    
    if not ffmpeg.ffprobe.video:
        raise ValueError("You did not request a video file")
    
    audio = ffmpeg.ffprobe.audio[0]
    video = ffmpeg.ffprobe.video[0]
    
    
    issues = {}
    file.write("----------{}---------- \n".format(original_name))

    
    file_format = path.split('.')[1]
    
    #Get audio information
    audio_codec = audio.__dict__.get('codec_name', '')
    audio_channels = audio.__dict__.get('channels', '')
    audio_bitrate = int(audio.__dict__.get('bit_rate', ''))
    
    if audio_codec != 'aac':
        issues["audio_codec"] = audio_codec
    
    if audio_bitrate > 256000:
        issues["audio_bitrate"] = audio_bitrate
        
    if audio_channels != '2':
        issues["audio_channels"] = audio_channels
    
    video_codec = video.__dict__.get('codec_name', '')
    video_width = video.__dict__.get('width', '')
    video_height = video.__dict__.get('height', '')
    video_fps = int(video.__dict__.get('framerate', ''))
    video_bitrate = int(video.__dict__.get('bit_rate', ''))
    
    if video_codec != 'h264':
        issues["video_codec"] = video_codec
    
    if video_width != '640':
        issues["video_width"] = video_width
        
    if video_height != '360':
        issues["video_height"] = video_height
        
    if video_fps != 25:
        issues["video_fps"] = video_fps
        
    if video_bitrate < 2000000 or video_bitrate > 5000000:
        issues["video_bitrate"] = video_bitrate
        
    for attr, value in issues.items():
        file.write("{}: {} \n".format(attr,value))
        
    
    ## return bitrate to desired value
    audio_bitrate = audio_bitrate if (audio_bitrate < 256000) else 256000
    video_bitrate = video_bitrate if (video_bitrate < 5000000 and video_bitrate > 2000000) else 2560000
    
    converted_file = ffmpeg.convert(path, converted_name, {
        'format': 'mp4',
        'audio': {
            'codec': 'aac',
            'bitrate': audio_bitrate,
            'channels': 2
        },
        'video': {
            'codec': 'h264',
            'width': 640,
            'height': 360,
            'fps': 25,
            'bitrate': video_bitrate
        }})
    
    
    for time in converted_file:
        print("Converting {} ...\r".format(time))
        
    print("Finished converting {} \n".format(path))

##Attempt to convert each one
file = open("Problems.txt", "w")
for path in files:
    ConvertToMP4(path, file)
    
file.close()

DEBUG:__main__:Executing ffmpeg with command: /usr/bin/ffmpeg -i films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4 -acodec aac -ac 2 -strict experimental -vcodec libx264 -r 25 -s 640x360 -aspect 640:360 -f mp4 -y films/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4


['00:00:02.60']
Converting 6 ...
['00:00:03.94']
Converting 9 ...
['00:00:05.29']
Converting 13 ...
['00:00:06.95']
Converting 17 ...
['00:00:08.29']
Converting 20 ...
['00:00:09.51']
Converting 23 ...
['00:00:10.62']
Converting 26 ...
['00:00:11.66']
Converting 29 ...
['00:00:13.01']
Converting 32 ...
['00:00:14.29']
Converting 35 ...
['00:00:15.95']
Converting 39 ...
['00:00:17.66']
Converting 44 ...


DEBUG:__main__:Executing ffmpeg with command: /usr/bin/ffmpeg -i films/Cosmos_War_of_the_Planets.mp4 -acodec aac -ac 2 -strict experimental -vcodec libx264 -r 25 -s 640x360 -aspect 640:360 -f mp4 -y films/Cosmos_War_of_the_Planets_formatOK.mp4


['00:00:19.39']
Converting 48 ...
Finished converting films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4 

['00:00:03.92']
Converting 9 ...
['00:00:05.63']
Converting 14 ...
['00:00:07.31']
Converting 18 ...
['00:00:09.04']
Converting 22 ...
['00:00:10.98']
Converting 27 ...
['00:00:12.97']
Converting 32 ...
['00:00:14.91']
Converting 37 ...
['00:00:16.68']
Converting 41 ...


DEBUG:__main__:Executing ffmpeg with command: /usr/bin/ffmpeg -i films/Last_man_on_earth_1964.mov -acodec aac -ac 2 -strict experimental -vcodec libx264 -r 25 -s 640x360 -aspect 640:360 -f mp4 -y films/Last_man_on_earth_1964_formatOK.mp4


['00:00:18.64']
Converting 46 ...
Finished converting films/Cosmos_War_of_the_Planets.mp4 

['00:00:02.94']
Converting 7 ...
['00:00:04.94']
Converting 12 ...
['00:00:06.95']
Converting 17 ...
['00:00:08.57']
Converting 21 ...
['00:00:10.30']
Converting 25 ...
['00:00:12.16']
Converting 30 ...
['00:00:13.95']
Converting 34 ...
['00:00:15.95']
Converting 39 ...


DEBUG:__main__:Executing ffmpeg with command: /usr/bin/ffmpeg -i films/The_Gun_and_the_Pulpit.avi -acodec aac -ac 2 -strict experimental -vcodec libx264 -r 25 -s 640x360 -aspect 640:360 -f mp4 -y films/The_Gun_and_the_Pulpit_formatOK.mp4


['00:00:17.96']
Converting 44 ...
Finished converting films/Last_man_on_earth_1964.mov 

['00:00:02.94']
Converting 7 ...
['00:00:04.45']
Converting 11 ...
['00:00:05.95']
Converting 14 ...
['00:00:07.95']
Converting 19 ...
['00:00:09.57']
Converting 23 ...
['00:00:11.58']
Converting 28 ...
['00:00:12.94']
Converting 32 ...
['00:00:14.95']
Converting 37 ...
['00:00:16.93']
Converting 42 ...
['00:00:18.94']
Converting 47 ...
Finished converting films/The_Gun_and_the_Pulpit.avi 



DEBUG:__main__:Executing ffmpeg with command: /usr/bin/ffmpeg -i films/The_Hill_Gang_Rides_Again.mp4 -acodec aac -ac 2 -strict experimental -vcodec libx264 -r 25 -s 640x360 -aspect 640:360 -f mp4 -y films/The_Hill_Gang_Rides_Again_formatOK.mp4


['00:00:03.45']
Converting 8 ...
['00:00:05.15']
Converting 12 ...
['00:00:07.19']
Converting 17 ...
['00:00:09.17']
Converting 22 ...
['00:00:11.00']
Converting 27 ...
['00:00:12.81']
Converting 32 ...
['00:00:14.76']
Converting 36 ...
['00:00:16.78']
Converting 41 ...
['00:00:18.57']
Converting 46 ...
Finished converting films/The_Hill_Gang_Rides_Again.mp4 

